In [1]:
# Import TensorFlow v2.
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

In [2]:
# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

# Training Parameters
learning_rate = 0.001
training_steps = 1000
batch_size = 32
display_step = 100

# Network Parameters
# MNIST image shape is 28*28px, we will then handle 28 sequences of 28 timesteps for every sample.
num_input = 28 # number of sequences.
timesteps = 28 # timesteps.
num_units = 32 # number of neurons for the LSTM layer.

In [3]:

# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, 28, 28]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [10]:
class BLSTM(Model):
    def __init__(self):
        super(BLSTM, self).__init__()

        self.lstm_fw = layers.LSTM(units=num_units)
        self.lstm_bw = layers.LSTM(units=num_units, go_backwards=True)
        self.bi_lstm = layers.Bidirectional(self.lstm_fw, backward_layer=self.lstm_bw)
        self.fc = layers.Dense(num_classes)

    def call(self, x, training=False):

        x = self.bi_lstm(x)
        x = self.fc(x)
        if not training:
            x = tf.nn.softmax(x)

        return x

bi_lstm = BLSTM()


In [11]:
# Cross-Entropy Loss.
# Note that this will apply 'softmax' to the logits.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Adam optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [15]:
def run_optimization(x, y):

    with tf.GradientTape() as tape:

        pred = bi_lstm(x, training=True)

        loss = cross_entropy_loss(pred, y)

    trainable_variables = bi_lstm.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [16]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = bi_lstm(batch_x, training=True)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 0.390075, accuracy: 0.875000
step: 200, loss: 0.179096, accuracy: 0.968750
step: 300, loss: 0.265596, accuracy: 0.906250
step: 400, loss: 0.311065, accuracy: 0.875000
step: 500, loss: 0.255913, accuracy: 0.875000
step: 600, loss: 0.373139, accuracy: 0.843750
step: 700, loss: 0.098541, accuracy: 0.968750
step: 800, loss: 0.326110, accuracy: 0.875000
step: 900, loss: 0.284465, accuracy: 0.937500
step: 1000, loss: 0.194979, accuracy: 0.968750
